In [ ]:
import pandas as pd # Pour le dataframe
from skimage.transform import resize
import numpy as np # Pour la normalisation et calculs de moyenne
import matplotlib.pyplot as plt # Pour la visualisation

from PIL import Image

import librosa # Pour l'extraction des features et la lecture des fichiers wav
import librosa.display # Pour récupérer les spectrogrammes des audio
import librosa.feature

import os # C'est ce qui va nous permettre d'itérer sur les fichiers de l'environnement de travail

from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split, GridSearchCV, validation_curve, RandomizedSearchCV # Split de dataset et optimisation des hyperparamètres
from sklearn.ensemble import RandomForestClassifier # Random forest
from sklearn.ensemble import GradientBoostingClassifier # XGBoost
from sklearn.neighbors import KNeighborsClassifier # k-NN
from sklearn.svm import SVC # SVM
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, f1_score, zero_one_loss, classification_report # Métriques pour la mesure de performances
from sklearn.preprocessing import normalize, StandardScaler

import tensorflow as tf # Pour le reseau de neurones simple et pour le CNN
from tensorflow.keras.models import load_model

import seaborn as sns

import keras
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from keras.optimizers import Adam
from keras.utils import to_categorical

from xgboost import XGBClassifier
from pprint import pprint

### Chargement du CSV

In [ ]:
df = pd.read_csv('./Data/1Mdata.csv')

In [ ]:
df.head()

##### Afficher le total de titres par genres :

In [ ]:
genre_titles_counts = df['genres'].value_counts()
print(genre_titles_counts)

##### Compter les genres uniques :

In [ ]:
unique_genres_count = df['genres'].nunique()
print(f"Number of unique genres: {unique_genres_count}")

##### Identification des genres minoritaires :
    Identifiez les genres qui ont un nombre très faible de titres. Vous pouvez fixer un seuil en dessous duquel les genres seront considérés comme trop minoritaires.

In [ ]:
threshold = 500  # Seuil
genre_counts = df['genres'].value_counts()
minority_genres = genre_counts[genre_counts <= threshold].index
print(minority_genres)

##### Quelles sont les implications concernant ce résultat ?
**1. Distribution très déséquilibrée :**
Notre ensemble de données est très déséquilibré, avec quelques genres très bien représentés et une grande majorité de genres avec peu de titres.

**2. Risque de surapprentissage :**
Avec une telle distribution, il y a un risque élevé que notre modèle soit biaisé en faveur des genres bien représentés.

In [ ]:
threshold = 2000  # Seuil
genre_counts = df['genres'].value_counts()
to_remove = genre_counts[genre_counts <= threshold].index
df = df[~df['genres'].isin(to_remove)]

In [ ]:
unique_genres_count = df['genres'].nunique()
print(f"Number of unique genres after operation: {unique_genres_count}")

##### Résultats après le tri
**1. Réduction significative :**
Nous avons considérablement réduit le nombre de genres, ce qui devrait faciliter l'entraînement du modèle et améliorer sa généralisation.

**2. Focus sur les genres populaires :**
Avec seulement 26 genres restants, votre modèle sera très axé sur les genres les plus populaires ou les plus couramment représentés dans notre ensemble de données.

**3. Risque de perte d'information :**
Il est important de noter que cette approche pourrait entraîner la perte d'informations sur les genres moins courants, ce qui pourrait être pertinent selon l'objectif de notre projet.

___

##### Affichage des genres restants, ainsi que du nombre de titres par genres

In [ ]:
remaining_genres = df['genres'].unique()
print("Remaining genres after applying the threshold:")
print(remaining_genres)

In [ ]:
remaining_genre_counts = df['genres'].value_counts()
print("Remaining genres and their counts after applying the threshold:")
print(remaining_genre_counts)

___

##### Equilibrage des classes :
**Sous-échantillonnage :**
Nous pouvons réduire le nombre d'échantillons des classes sur-représentées pour les équilibrer avec les classes moins représentées.

In [ ]:
# Nombre d'échantillons à conserver pour la classe "Rock"
n_samples_rock = 60000  # Vous pouvez ajuster ce nombre selon vos besoins

# Séparer les données de la classe "Rock" et des autres classes
df_rock = df[df['genres'] == 'Rock']
df_other = df[df['genres'] != 'Rock']

# Sous-échantillonnage de la classe "Rock"
df_rock_sampled = df_rock.sample(n=n_samples_rock, random_state=1)

# Fusionner les données sous-échantillonnées de "Rock" avec les autres données
df_balanced = pd.concat([df_rock_sampled, df_other])

# Vérifier la nouvelle distribution des classes
new_genre_counts = df_balanced['genres'].value_counts()
print("New genre distribution:")
print(new_genre_counts)

**Supprimer "Soundtrack" :**
Étant donné que "Soundtrack" peut être un mélange de différents genres et styles, il peut être difficile pour le modèle de le classer de manière significative. Le supprimer pourrait simplifier le problème de classification.

**Fusionner "Other" et "Easy Listening" :**
Si "Other" est une catégorie fourre-tout et que "Easy Listening" est un genre moins représenté, les fusionner peut aider à équilibrer les classes tout en réduisant le nombre de catégories à prédire.

In [ ]:
# Supprimer les lignes où le genre est "Soundtrack"
df_balanced = df_balanced[df_balanced['genres'] != 'Soundtrack']

# Fusionner les genres "Other" et "Easy Listening" en une seule catégorie "Other/Easy Listening"
df_balanced['genres'] = df_balanced['genres'].apply(lambda x: 'Other/Easy Listening' if x in ['Other', 'Easy Listening'] else x)

# Vérifier la nouvelle distribution des genres
new_genre_counts = df_balanced['genres'].value_counts()
print("New genre distribution after adjustments:")
print(new_genre_counts)


___

Mainteanant que notre CSV à l'air relativement exploitable, nous allons le sauvegarder

In [ ]:
# Enregistrer le DataFrame dans un nouveau fichier CSV
df_balanced.to_csv('balanced_genre_dataset.csv', index=False)

In [ ]:
df = pd.read_csv('./Data/balanced_genre_dataset.csv')

In [ ]:
unique_genres_count = df['genres'].nunique()
print(f"Number of unique genres after operation: {unique_genres_count}")

___

#### Création du modèle IA de forêts aléatoires avec sklearn
Les forêts aléatoires sont un excellent choix pour un modèle basé sur des métadonnées. Ce sont des modèles d'ensemble qui sont généralement bien adaptés pour gérer des jeux de données de grande dimension et des caractéristiques hétérogènes. Ils sont également moins sensibles au surajustement par rapport à un arbre de décision unique.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
data = pd.read_csv('./Data/balanced_genre_dataset.csv')

In [ ]:
X = data.drop(['genres', 'track_id', 'song_id'], axis=1)  # Supprimer les colonnes inutiles à l'entraînement du modèle
y = data['genres']  # Utiliser la colonne des genres comme étiquettes

In [ ]:
X.head()

In [ ]:
# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Créer le modèle de forêt aléatoire
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Entraîner le modèle
clf.fit(X_train, y_train)

# Faire des prédictions
y_pred = clf.predict(X_test)

# Évaluer le modèle
print("Accuracy:", accuracy_score(y_test, y_pred))

___

Test d'un modèle de classification à travers un Réseau de Neurones

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalisation des données
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Créer le modèle de réseau de neurones
mlp = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=300, learning_rate_init=0.001)

# Entraîner le modèle
mlp.fit(X_train, y_train)

# Faire des prédictions
y_pred = mlp.predict(X_test)

# Évaluer le modèle
print("Accuracy:", accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.model_selection import cross_val_score

# Utilisation de la validation croisée
scores = cross_val_score(mlp, X, y, cv=5)
print("Cross-Validation Scores: ", scores)
print("Mean Cross-Validation Score: ", scores.mean())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Calcul de la matrice de confusion
cm = confusion_matrix(y_test, y_pred)

# Affichage de la matrice de confusion
plt.figure(figsize=(10, 6))
sns.heatmap(cm, annot=True, fmt="d", cbar=False)
plt.title("Matrice de confusion")
plt.xlabel("Prédiction")
plt.ylabel("Vérité")
plt.show()

Commme nous pouvons le voir, les scores ne sont pas convaiquants

In [ ]:
import pickle

In [ ]:
# Enregistrement du modèle
filename = 'BadBoyModelV2.sav'
pickle.dump(mlp, open(filename, 'wb'))

In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
round(loaded_model.score(X_test, y_test), 2)

Nous n'avons pas de pistes pour exploiter le modèle et faire une prédiction avec ce dernier

___

## Conclusions :
- Il n'est pas possible de faire des prédictions avec un tel modèle
- Le modèle manque de précision dû au fait qu'il a été entrainé sur des métadonnées et non des données riches venant de fichiers audios

## Axes d'Améliorations : 
- Télécharger plusieurs datasets de [fichiers audios](https://music-classification.github.io/tutorial/part2_basics/dataset.html), les regrouper et entrainer le modèle sur ces derniers au lieu de se baser sur des **csv**
- Faire une prédiction sur le spectrogramme obtenu du fichier audio

___

#### Points à voir pour la V3
1. Vérification de la qualité des données fournies
2. Poids des données (si possible)
3. Ré-entraînement du modèle